In [1]:
import requests
import time
import hashlib
import hmac
import uuid
import os

In [8]:
def read_api_keys(test=0):
    
	key_file = 'AK.txt'
    
	if test:
		key_file= 'TAK.txt'
    
	current_dir = os.path.abspath(os.path.dirname(''))
	proj_dir = os.path.abspath(os.path.join(current_dir, '..', '..'))
	# proj_dir = os.path.dirname(os.path.abspath(__file__))
    
	file_path = os.path.join(proj_dir, key_file)
    
	keys = {}
    
	try:
		with open(file_path, 'r') as file:
			for line in file:
				if '=' in line: 
					key, value = line.strip().split('=', 1)
					keys[key] = value

	except FileNotFoundError:
		print(f"Error: File not found at {file_path}")
	except Exception as e:
		print(f"Error: {e}")
		
	return keys

keys = read_api_keys(test=1)
public_key = keys.get('PUBLIC_KEY')
private_key = keys.get('PRIVATE_KEY')


In [34]:
httpClient=requests.Session()
recv_window=str(5000)	# Time limit for requests
url="https://api-testnet.bybit.com" # Testnet endpoint (Make sure using testnet api keys)

def HTTP_Request(endPoint,method,payload,Info):
    global time_stamp
    time_stamp=str(int(time.time() * 10 ** 3))
    signature=genSignature(payload)
    headers = {
        'X-BAPI-API-KEY': public_key,
        'X-BAPI-SIGN': signature,
        'X-BAPI-TIMESTAMP': time_stamp,
        'X-BAPI-RECV-WINDOW': recv_window,
    }
    if(method=="POST"):
        response = httpClient.request(method, url+endPoint, headers=headers, data=payload)
    else:
        response = httpClient.request(method, url+endPoint+"?"+payload, headers=headers)
    
    return response.json()

def genSignature(payload):
    param_str= str(time_stamp) + public_key + recv_window + payload
    hash = hmac.new(bytes(private_key, "utf-8"), param_str.encode("utf-8"),hashlib.sha256)
    signature = hash.hexdigest()
    return signature

In [45]:
#Create Order
endpoint="/v5/order/create"
method="POST"
params='{"category":"linear","symbol": "BTCUSDT","side": "Buy","positionIdx": 0,"orderType": "Limit","qty": "0.001","price": "10000"}'
HTTP_Request(endpoint,method,params,"Create")

{'retCode': 0,
 'retMsg': 'OK',
 'result': {'orderId': 'ce6d3129-e6a7-4811-ade2-42e81aa1afc8',
  'orderLinkId': ''},
 'retExtInfo': {},
 'time': 1734949128996}

In [46]:
#Get unfilled Orders
endpoint="/v5/order/realtime"
method="GET"
params='category=linear&settleCoin=USDT'
response = HTTP_Request(endpoint,method,params,"UnFilled")
print(response['result']['list'])
order_id = response['result']['list'][0]['orderId']

[{'symbol': 'BTCUSDT', 'orderType': 'Limit', 'orderLinkId': '', 'slLimitPrice': '0', 'orderId': 'ce6d3129-e6a7-4811-ade2-42e81aa1afc8', 'cancelType': 'UNKNOWN', 'avgPrice': '', 'stopOrderType': '', 'lastPriceOnCreated': '94600', 'orderStatus': 'New', 'createType': 'CreateByUser', 'takeProfit': '', 'cumExecValue': '0', 'tpslMode': '', 'smpType': 'None', 'triggerDirection': 0, 'blockTradeId': '', 'isLeverage': '', 'rejectReason': 'EC_NoError', 'price': '10000', 'orderIv': '', 'createdTime': '1734949128996', 'tpTriggerBy': '', 'positionIdx': 0, 'timeInForce': 'GTC', 'leavesValue': '10', 'updatedTime': '1734949128999', 'side': 'Buy', 'smpGroup': 0, 'triggerPrice': '', 'tpLimitPrice': '0', 'cumExecFee': '0', 'leavesQty': '0.001', 'slTriggerBy': '', 'closeOnTrigger': False, 'placeType': '', 'cumExecQty': '0', 'reduceOnly': False, 'qty': '0.001', 'stopLoss': '', 'marketUnit': '', 'smpOrderId': '', 'triggerBy': ''}]


In [47]:
#Cancel Order
endpoint="/v5/order/cancel"
method="POST"
params='{"category":"linear","symbol": "BTCUSDT","orderId": "'+order_id+'"}'
HTTP_Request(endpoint,method,params,"Cancel")

{'retCode': 0,
 'retMsg': 'OK',
 'result': {'orderId': 'ce6d3129-e6a7-4811-ade2-42e81aa1afc8',
  'orderLinkId': ''},
 'retExtInfo': {},
 'time': 1734949154740}